# 利用TOF中的$\gamma$峰，对TOF进行绝对刻度（找到飞行时间零点）

- 目的是将切线探测器修正成环形；
- 因为，实验上中子能量$E_n$与TOF关关联，及相同能量的中子，飞行相同的距离其时间相等。但本实验探测器由于式切线形状，相同能量中子由靶点到中心和两端的距离不同，这样TOF不仅与能量有关，还与距离有关。为此，需要TOF绝对的刻度，即修正探测器形状；
- 最终目的，TOF仅与能量相关，TOF直接代表能量；

## 打开root文件，继承tree中Branch。

In [1]:
%jsroot on
TCanvas *c1=new TCanvas("c1","c1");//创建画布

//声明branch变量
Double_t ctof,tof;
Int_t pid;
Double_t tu, td, x;

//打开root文件
TFile *ipf=new TFile("./tree.root");//打开ROOT文件
//得到tree的指针
TTree *tree=(TTree*)ipf->Get("tree");

//将tree中Branch与变量地址联系
tree->SetBranchAddress("ctof",&ctof);
tree->SetBranchAddress("tof",&tof);
tree->SetBranchAddress("pid",&pid);
tree->SetBranchAddress("tu",&tu);   
tree->SetBranchAddress("td",&td);
tree->SetBranchAddress("x",&x);

## 模拟计算的TOF

In [2]:
tree->Draw("ctof>>(1000,20,120)");
c1->Draw();

## 查看$\gamma$的TOF与位置关联

In [3]:
tree->Draw("ctof:td-tu>>(2000,-20,50,50,40,45)","pid==0","colz");//加入条件,gamma
c1->Draw();

- 上图为$\gamma$事件，其速度为光速，定值；
- 两端高中间底，速度相同，越远离中心，飞行距离越长长，耗时越大；

## 逐事件提取$\gamma$的ctof事件，ctof进行飞行距离的修正

In [4]:
TH2D *hgtofx=new TH2D("hgtofx","hgtofx",100,-120,120,100,39,48);
TH1D *hgctof=new TH1D("hgctof","hgctof",100,39,48);//挑出打在探测器中心的gamma事件
TH2D *ctofvstx=new TH2D("ctofvstx","ctofvstx",240,-120,120,100,41,45);//挑出打在探测器全区域的gamma事件


//对事件进行遍历 
Long64_t nentries=tree->GetEntries();
for(Long64_t jentry=0; jentry<nentries; jentry++) 
    {
    tree->GetEntry(jentry);
    Double_t tx=3.742*(td-tu-14.915);  //时间差法刻度的位置
    if(ctof>42&& ctof<44.5) 
      {   
       Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);//考虑了探测器的厚度的飞行距离
       Double_t ctofa=(ctof)/d*500.;//不同距离的tof统一修正到500 cm
       hgtofx->Fill(tx,ctofa);
       ctofvstx->Fill(tx,ctof);
       if(abs(tx)<5) hgctof->Fill(ctof);//挑出打在探测器中心的事件
      }
    }
hgtofx->Draw("colz");
c1->Draw();


- 修正之后发现依然与位置相关，修正后（$TOFc=\frac{TOF}{d_x}*d_0$）两端飞行时间比中间小;
- 意味着两端修正过大？？？

## 修正offset的影响

### 以$\gamma$打在探测器中心时的TOF值为修正标准

In [5]:
hgctof->Draw();
hgctof->Fit("gaus");
c1->Draw();

 FCN=5.9184 FROM MIGRAD    STATUS=CONVERGED      58 CALLS          59 TOTAL
                     EDM=1.21945e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     1.83370e+02   5.78633e+00   5.99719e-03  -1.00487e-04
   2  Mean         4.29564e+01   7.74609e-03   2.04832e-05   4.86895e-04
   3  Sigma        3.02451e-01   5.70756e-03   6.52101e-06  -7.91675e-02


### 以$\gamma$打在探测器全区域时的TOF值为修正标准

In [6]:
ctofvstx->Draw("colz");
ctofvstx->Fit("pol2");
c1->Draw();


****************************************
Minimizer is Linear
Chi2                      =  1.74542e+06
NDf                       =         6961
p0                        =      42.9542   +/-   0.00549166  
p1                        =  1.95083e-06   +/-   6.33681e-05 
p2                        =  3.20798e-05   +/-   1.22864e-06 


## TOF的绝对刻度
虽然光在塑闪传播时间可根据模拟得到，但电子学、线缆、PMT导致的offset成分无法确定。

### 确定offset(包含塑闪、电子学、PMT等因素）方法-以$\gamma$打在探测器中心为基准修正
- $t_u = TOF + (L - x)/v_{sc} + T_{uoff}$
- $t_d = TOF + (L + x)/v_{sc} + T_{doff}$
- $TOF =(t_u+t_d)/2-L/v_{sc}-( T_{uoff}+ T_{doff})/2$; $C=-L/v_{sc}-( T_{uoff}+ T_{doff})/2$

以光子打在探测器中心为标准，$TOF=d/c$，带入$d = 502.5 cm$，$c=3.0*10^{8} m/s$, 可得$TOF = 16.748~ns$；

- 拟合$hgctof(0) = 42.9576$,$ C= TOF - hgctof(0) =-26.2096~ns$;

In [7]:
TH2D *hgtofcx=new TH2D("hgtofcx","corrected TOF",100,-120,120,100,15,19);//实验测量抽取的tof修正到500 cm（考虑offset）与位置关联
TH2D *hgtofcx1=new TH2D("hgtofcx1","corrected TOF (all gamma)",100,-120,120,100,15,19);//实验测量抽取的tof修正到500 cm（考虑offset_all gamma）与位置关联

TH1D *htofc=new TH1D("htofc","htofc",200,0,100);//实验测量抽取的tof修正到500 cm，即考虑offset；
TH1D *htofc1=new TH1D("htofc1","htofc (all gamma)",200,0,100);//实验测量抽取的tof修正到500 cm，即考虑offset；
TH1D *htof=new TH1D("htof","htof",200,0,100);//模拟中真实的tof修正到500 cm

In [8]:
//Long64_t nentries=tree->GetEntries();
for(Long64_t jentry=0; jentry<nentries; jentry++) {
    tree->GetEntry(jentry);
    Double_t tx=3.742*(td-tu-14.915);
    Double_t d=TMath::Sqrt(502.5*502.5+tx*tx);
    
    Double_t tofc=(ctof-26.2096)/d*500.;//normalized to 500cm
    
    Double_t C=d/30.-(42.9542+1.95083e-6*tx+3.20798e-5*tx*tx);
    Double_t tofc1=(ctof+C)/d*500.;//normalized to 500cm by  all gamma
    
    hgtofcx->Fill(tx,tofc);//gamma hits the center of the det.
    hgtofcx1->Fill(tx,tofc1);//gamma hits the any position of the det.
    htofc->Fill(tofc);
    htofc1->Fill(tofc1);
    htof->Fill(tof*500./d);
  }
hgtofcx->Draw("colz");//tofc与x之间无关联
c1->Draw();

- 为何不考虑offset影响会向下弯？
- 疑似$\frac{TOF}{d_x}*d_0$的修正，越偏离中心，修正越大；
- 当加入offset后有$（\frac{TOF}{d_x}-\frac{C}{d_x}）*d_0$，即在原有的修正基础上填加了一个附加项，而这个附加项使得越偏离中心位置，附加修正越小，弥补了之前边缘修正过大的问题。

In [9]:
c1->SetLogy();
htofc->Draw();//修正后的飞行时间谱
htof->SetLineColor(kRed);
htof->Draw("same");
c1->Draw();

实验测量抽取的tof修正到500 cm（考虑offset）与模拟中真实的tof修正到500 cm符合很好！

### 确定offset(包含塑闪、电子学、PMT等因素）方法-以$\gamma$
同上：
- $t_u = TOF + (L - x)/v_{sc} + T_{uoff}$
- $t_d = TOF + (L + x)/v_{sc} + T_{doff}$
- $TOF =(t_u+t_d)/2-L/v_{sc}-( T_{uoff}+ T_{doff})/2$; $C=-L/v_{sc}-( T_{uoff}+ T_{doff})/2$

以光子打在探测器任意位置，$TOF(tx)=\frac{\sqrt{d^{2}+x^{2}}}{c}$，其中$d = 502.5~cm$，$c=3.0*10^{8}~ m/s=30~cm/ns$；

- 拟合cfotvstx可得hgctof(tx),则 C(tx)= TOF(tx) - hgctof(tx) ;

In [10]:
c1->SetLogy(0);
hgtofcx1->Draw("colz");//tofc1与x之间无关联
c1->Draw();

In [11]:
c1->SetLogy();
htofc1->Draw();//修正后的飞行时间谱
htof->SetLineColor(kRed);
htof->Draw("same");
c1->Draw();

In [12]:
ipf->Close();//关闭root文件

In [13]:
!jupyter nbconvert homework1.1-3 --to html

[NbConvertApp] Converting notebook homework1.1-3.ipynb to html

[NbConvertApp] Writing 723164 bytes to homework1.1-3.html

